<h4>
Essa tarefa tem como objetivo demonstrar uma Pipeline de Supervisão Fraca.<br>
Para essa tarefa eu escolhi utilizar o conjunto de dados da minha primeira tarefa no meu estágio, que comecei a um ano atrás.<br>
Essa tarefa tem o objetivo encontrar as espécies, o tipo do corte e as dimensões das descrições de madeira.<br>
Além de ser um trabalho de MLP, que na época eu não tinha nenhuma experiência, também teve a grande dificuldade dos dados não estarem rotulados.<br>
A minha solução final envolveu a construção de um conjunto de treino e validação manual e utilizar uma LLM com um schema definido para retornar a espécie, o corte e as dimensões.
</h4>

<h4>
Temos algumas características que tornaram esse problema difícil para uma LLM, o primeiro, as descrições de produto não são estruturadas, além de não existir nenhum tipo de ordem para cada uma das classes de interesse também podemos ter casos em que essas classes estão faltando, como por exemplo uma descrição possuir a espécie, o corte, mas não possuir a dimensão.<br>
Exemplo: "tabua de massaranduba."<br>
Um outro problema são as situações em que uma descrição possui múltiplos de uma mesma classe.<br>
Exemplo: "tabua/viga de massaranduba."<br>
Um problema mais simples são erros ortográficos, sinônimos e nomes ciêntificos da espécie, existem múltiplas maneiras de chamar um mesmo produto que faz sentido no contexto de produtos de madeira, mas são palavras completamente diferentes em outros contextos.<br>
Exemplo1: "tabua de cetim."<br>
Exemplo2: "tabua de pau amarelo." -- Sinônimo<br>
Exemplo3: "tabua de cedrelinga cateniformis." -- Nome Ciêntifico<br>
Além de problemas intrísecos ao Dataset, como as descrições de outros produtos que não são madeira com o NCM na nota fiscal como produto de madeira. Esses foram as maiores dificuldades que impactaram a qualidade da solução utilizanção LLM.
</h4>

<h4>
Essa atividade se mostrou uma boa oportunidade para revisitar esse problema.<br>
Agora que tenho mais conhecimento na área de NLP, como eu posso solucionar essa tarefa?<br>
Para isso eu vou partir do ponto em que eu reduzi o meu Dataset não rotulado para manter apenas as descrições únicas.<br>
Essas descrições já passaram por um processo de pré-processamento simples, como: tornar tudo minúsculo e substituir acentos pelas letras normais.<br>
Esse conjunto inicial é "madeiraDescricao.csv" e eu optei por manter apenas a coluna textual para evitar compartilhar dados sensíveis da nota.<br>
Como os meus dados não estão rotulados essa é uma ótima oportunidade para utilizar o que aprendi com Weak Learning.<br>
Primeiro preciso criar um conjunto de dados rotulados que será dividido em validação e teste. Mas para criar esse conjunto eu primeiro preciso entender como resolverei esses problemas.
</h4>

<h4>
A solução que decidi seguir foi a implementação de uma tarefa de NER, em que as entidades serão as <mark>ESPECIE</mark>, <mark>CORTE</mark> E <mark>DIMENSAO</mark>.<br>
Eu escolhi selecionar essa solução devido aos problemas mencionados, as multiplicidade das entidades, ou falta de alguma delas não seria um problema, além disso acredito que os problemas de sinônimos e nomes cientificos também são de fácil solução para essa tarefa.<br>
Então para começar eu selecionei uma amostra aleatória de 600 instâncias e as removi do conjunto não rotulado, para fazer o rótulo dessas entidades eu optei por utilizar o Argilla devido a possibilidade de subir um container localmente, evitando o vazamento desses dados, uma etapa que seria necessária no meu ambiente do estágio.<br>
O conjunto de dados que será rotulado é "labedMadeiraDescricao.csv" e o restante está no conjunto "unlabeledMadeiraDescricao.csv".<br>
Esse foi o resultado final 564 dos 600 erão produtos de madeira e foram rotulados, o restante não será utilizado.<br>
<img src="images/argilla.jpg" alt="Argilla" width="400"><br>
Agora que temos o conjunto de avaliação rotulado, "labeledTokenClassification.csv" podemos começar a resolver a tarefa.
</h4>

In [50]:
import pandas as pd

pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_rows', 100)

df_val_test = pd.read_csv('dataset/labeledTokenClassification.csv')
df_unlabeled = pd.read_csv('dataset/unlabeledMadeiraDescricao.csv')

In [51]:
## Test primeiros 100
df_unlabeled = df_unlabeled[:100]
len(df_unlabeled)

100

<h4>
Selecionar apenas as instâncias com o status de completo, para evitar utilizar as descrições que não são produtos de madeira.
</h4>

In [52]:
df_val_test = df_val_test[df_val_test.status == 'completed']

<h4>
Vamos separar em dois conjuntos, um para validação e outro para teste.
</h4>

In [53]:
from sklearn.model_selection import train_test_split

df_val, df_test = train_test_split(
    df_val_test,
    test_size=0.5,       
    random_state=1, # Necessário a mudança caso utilizar o conjunto de teste
    shuffle=True         
)
print(len(df_val), len(df_test))

282 282


In [54]:
len(df_unlabeled)

100

## Conversão para Objetos do Spacy

In [55]:
import skweak
import spacy

nlp = spacy.load("pt_core_news_lg")

In [56]:
spacy_docs_train = list(nlp.pipe(df_unlabeled["text"].values))

spacy_docs_dev = list(nlp.pipe(df_val["text"].values))

spacy_docs_test = list(nlp.pipe(df_test["text"].values))

skweak.utils.docbin_writer(spacy_docs_train, "artefatos/spacy_docs_train.bin")
skweak.utils.docbin_writer(spacy_docs_dev, "artefatos/spacy_docs_dev.bin")
skweak.utils.docbin_writer(spacy_docs_test, "artefatos/spacy_docs_test.bin")

Write to artefatos/spacy_docs_train.bin...done
Write to artefatos/spacy_docs_dev.bin...done
Write to artefatos/spacy_docs_test.bin...done


In [21]:
# Load the training documents
spacy_docs_train = skweak.utils.docbin_reader(
    "artefatos/spacy_docs_train.bin", spacy_model_name="pt_core_news_lg"
)

# Load the validation documents
spacy_docs_dev = skweak.utils.docbin_reader(
    "artefatos/spacy_docs_dev.bin", spacy_model_name="pt_core_news_lg"
)

# Load the test documents
spacy_docs_test = skweak.utils.docbin_reader(
    "artefatos/spacy_docs_dev.bin", spacy_model_name="pt_core_news_lg"
)

# Convert the loaded documents to lists
# This step ensures that the documents are in a list format, which is easier to work with in subsequent steps
spacy_docs_train = list(spacy_docs_train)
spacy_docs_dev = list(spacy_docs_dev)
spacy_docs_test = list(spacy_docs_test)

In [22]:
spacy_docs_train[:2]

[angelim verm aproveitamento ripas, caibro mixto 4,5mt]

## 1.Criação de funções de rotulagem

<h4>
Vamos precisar fazer funções de rotulagem para cada uma das entidades de interesse, <mark>ESPECIE</mark>, <mark>CORTE</mark>, <mark>DIMENSAO</mark>.<br>
Para as funções de rotulagem de ESPECIE, podemos começar com o seu nome popular e sinônimos. Eu já tenho um dicionário que desenvolvi na época com as espécies mais relavantes para o nosso domínio no mercado do estado do RN.<br>
Por isso irei partir dele e adicionar alguns casos simples para compensar a falta da função de Levenshtein que utilizei na época para aceitar pequenas alterações na variação das palavras.
</h4>

In [23]:
madeiras_dicionario = {
    'acacia': ['angico'],
    'acapu': ['angelim de folha larga'],
    'amapa': [],
    'amescla': ['amesclao', 'breu', 'morcegueira', 'sucuruba'],
    'andiroba': ['angirova', 'carapa'],
    'angelim amargoso': ['angelim amarelo', 'fava', 'fava amargosa', 'faveira'],
    'angelim pedra': ['angelim verm', 'angelim vermelho', 'angelim falso', 'faveira ferro'],
    'angelim indefinido': ['angelim', 'angelin', 'ang', 'ange'],
    'cedrinho': ['quaruba', 'quaruvatinga', 'mandioqueira', 'quaruba vermelha'],
    'cedro': [],
    'cerejeira': [],
    'cetim': ['pau amarelo', 'pau cetim', 'piquia cetim', 'garapa', ' muirajuba'],
    'coco pau': ['baru'],
    'copaiba': ['cupauba', 'copauva'],
    'cumaru': ['cumaru de folha grande', 'cumbar', 'cumbaru roxo', 'cumbaru'],
    'cupiuba': ['copiuba', 'copiuva'],
    'eucalipto': ['euc', 'ecalipto', 'eucalipito'],
    'freijo': [],
    'garapeira': [],
    'goiabao': ['abiu', 'abiurana'],
    'guajara': ['currupixa', 'angico-bravo', 'angico-cedro', 'angico-rosa', 'curupia'],
    'ipe': [],
    'itauba': [],
    'jatoba': ['jatai', 'utai', 'jati'],
    'jequitiba': ['cachimbeiro', 'estopeira', 'coatinga', 'bingueiro'],
    'louro amarelo': ['louro pardo, louro abacate', 'louro inhamui'],
    'louro canela': ['louro rosa', 'louro gamela', 'louro vermelho'],
    'louro indefinido': ['louro'],
    'louro preto': [],
    'marupa': [],
    'massaranduba': ['macaranduba', 'masaranduba', 'maparajuba', 'aparaui', 'paraju', 'mass', 'massa'],
    'matamata': [],
    'melanceira': ['sucupira-pepino'],
    'mogno': [],
    'muiracatiara': [],
    'para para': ['morototo', 'mandioqueira'],
    'pau louro': [],
    'pau mulato': [],
    'peroba': ['amapa amargoso', 'parahancornia'],
    'pinho': ['araucaria'],
    'pinus': ['pinheiro', 'pinnus'],
    'piquia': ['piqui', 'piquiarana', 'pequia'],
    'roxinho': ['pau roxo', 'guarabu', 'pau ferro'],
    'sapucaia': ['jarana'],
    'sucupira': ['cutiba', 'sapupira', 'sebepira'],
    'sumauma': ['ceiba'],
    'tanimbuca': ['amarelao', 'mirindiba', 'embiridiba', 'cuiarana ,carana', 'mangue'],
    'tatajuba': ['amarelo', 'amarelinho', 'garrote', 'bagaceira'],
    'tauari': ['imbirema', 'estopeiro', 'toari'],
    'taxi': [],
    'timborana': ['timbauba', 'angico'],
    'uxi': ['axua', 'cumate', 'paruru'],
    'mista': ['misto', 'mixta', 'mix'],
    'virola': ['ucuuba']
}

<h4>
Além dos nomes populares e sinônimos também podemos fazer funções de rotulagem para a entidade ESPECIE utilizando o nome ciêntifico da espécie de madeira.<br>
Essas funções não serão tão valiosas pela característica das descrições. Normalmente quando o nome ciêntifico está na descrição do produto ele está acompanhado do nome popular.<br>
Mesmo assim eu achei relevante coletar essas entidades, porém irei utilizar um outro dicionário para tal, dessa forma poderemos diferenciar qual a fonte de rotulagem para a ESPECIE.
</h4>

In [24]:
cientifico_madeiras_dicionario = {
    'acapu': ['vouacapoua americana', 'vouacapoua'],
    'amapa': ['parahancornia amapa', 'parahancornia'],
    'amescla': ['protium heptaphyllum', 'protium', 'heptaphyllum'],
    'andiroba': ['carapa guianensis', 'carapa', 'guianensis'],
    'angelim amargoso': ['vatairea sericea', 'vatairea', 'sericea'],
    'angelim pedra': ['hymenolobium petraeum', 'hymenolobium', 'petraeum'],
    'angelim indefinido': ['dinizia excelsa', 'dinizia', 'excelsa'],
    'cedrinho': ['erisma uncinatum', 'erisma', 'uncinatum'],
    'cedro': ['cedrela fissilis', 'cedrela', 'fissilis'],
    'cerejeira': ['amburana cearensis', 'amburana', 'cearensis'],
    'cetim': ['cedrelinga cateniformis', 'cedrelinga', 'cateniformis'],
    'coco pau': ['orbygnia phalerata', 'orbygnia', 'phalerata'],
    'copaiba': ['copaifera langsdorffii', 'copaifera', 'langsdorffii'],
    'cumaru': ['dipteryx odorata', 'dipteryx', 'odorata'],
    'cupiuba': ['goupia glabra', 'goupia', 'glabra'],
    'eucalipto': ['eucalyptus'],
    'freijo': ['cordia goeldiana', 'cordia', 'goeldiana'],
    'garapeira': ['apuleia leiocarpa', 'apuleia', 'leiocarpa'],
    'goiabao': ['qualea'],
    'guajara': ['terminalia amazonia', 'terminalia', 'amazonia'],
    'ipe': ['handroanthus'],
    'itauba': ['mezilaurus itauba', 'mezilaurus', 'itauba'],
    'jatoba': ['hymenaea courbaril', 'hymenaea', 'courbaril'],
    'jequitiba': ['cariniana legalis', 'cariniana', 'legalis'],
    'louro amarelo': ['ocotea'],
    'louro canela': ['nectandra'],
    'louro indefinido': ['ocotea'],
    'louro preto': ['ocotea porosa', 'ocotea', 'porosa'],
    'marupa': ['simarouba amara', 'simarouba', 'amara'],
    'massaranduba': ['manilkara huberi', 'manilkara', 'huberi'],
    'matamata': ['eschweilera'],
    'melanceira': ['trattinnickia burserifolia', 'trattinnickia', 'burserifolia'],
    'mogno': ['swietenia macrophylla', 'swietenia', 'macrophylla'],
    'muiracatiara': ['astronium lecointei', 'astronium', 'lecointei'],
    'para para': ['jacaranda copaia', 'jacaranda', 'copaia'],
    'pau louro': ['ocotea'],
    'pau mulato': ['calycophyllum spruceanum', 'calycophyllum', 'spruceanum'],
    'peroba': ['aspidosperma'],
    'pinho': ['araucaria angustifolia', 'araucaria', 'angustifolia'],
    'pinus': ['pinus'],
    'piquia': ['caryocar villosum', 'caryocar', 'villosum'],
    'roxinho': ['peltogyne'],
    'sapucaia': ['lecythis pisonis', 'lecythis', 'pisonis'],
    'sucupira': ['bowdichia virgilioides', 'bowdichia', 'virgilioides'],
    'sumauma': ['ceiba pentandra', 'ceiba', 'pentandra'],
    'tanimbuca': ['buchenavia tetraphylla', 'buchenavia', 'tetraphylla'],
    'tatajuba': ['bagassa guianensis', 'bagassa', 'guianensis'],
    'tauari': ['couratari'],
    'taxi': ['sclerolobium'],
    'timborana': ['enterolobium schomburgkii', 'enterolobium', 'schomburgkii'],
    'uxi': ['endopleura uchi', 'endopleura', 'uchi'],
    'virola': ['virola']
}

<h4>
Para a entidade de <mark>CORTE</mark> podemos fazer um processo bem similar ao do primeiro dicionário de madeira, utilizando o nome popular do tipo de corte e o seu sinônimo.<br>
Similar ao caso de dicionário também teremos que compensar pelo fator de que eu não estou usando Levenshtein, por isso precisamos aceitar erros comuns
</h4>

In [25]:
apresentacoes_dicionario ={
                            'alizar': ['alisar'],
                            'barrote':[],
                            'caibro':['caibrinho'],
                            'frechal': [],
                            'janela': ['janelao'],
                            'linha':[],
                            'mourao': [],
                            'quadrado': ['quadradro', 'bloco', 'file', 'quadradinho'],
                            'piquete': [],
                            'pontalete': [],
                            'porta': ['portal', 'aduela'],
                            'prancha':['pranchoes', 'pranchinha', 'pranchao'],
                            'ripa':['ripao'],
                            'tabua':['tauba'],
                            'taipa':[],
                            'sarrafo':['serrafo'],
                            'viga':['virga','vigota', 'poste']
                            }

<h4>
E também irei complementar com o plural dos tipos de corte que terminam em vogal.
</h4>

In [26]:
def adc_plural(apresentacoes_dict):
    updated_dict = {}
    for key, synonyms in apresentacoes_dict.items():
        pluralized_synonyms = []
        
        if key[-1].lower() in{'a', 'e', 'i', 'o', 'u'}:
            plural = key + "s"
        
            if plural not in synonyms:
                pluralized_synonyms.append(plural)

        for term in synonyms:
            pluralized_synonyms.append(term)
            
            if term and term[-1].lower() in {'a', 'e', 'i', 'o', 'u'}:
                if " " in term:
                    parts = term.split()
                    parts[-1] += "s"
                    plural = " ".join(parts)
                else:
                    plural = term + "s"
                pluralized_synonyms.append(plural)
        
        seen = set()
        unique_synonyms = [t for t in pluralized_synonyms 
                          if not (t in seen or seen.add(t))]
        
        updated_dict[key] = unique_synonyms
        
    return updated_dict

apresentacoes_dicionario = adc_plural(apresentacoes_dicionario)

<h4>
A coleta das dimensões do produto de madeira foi um dos maiores desafios, porque as descrições normalmente não seguem um padrão, o contribuinte quando escreve essa informação muitas vezes não coloca a dimensão associada a um número, como por exemplo "linha massaranduba 5 x 12", conseguimos assumir logicamente ao olhar para esse exemplo que não será 5 por 12 metros e muito provavelmente essa medida está em centímetros, existem alguns exemplos em que essa dimensão é ambígua, mas no geral conseguimos inferir.<br>
Além disso, outro problema é que esses números podem estar soltos no texto, como em descrições do tipo "linha 5x28 cm angelim c/ 6,0 mt", então precisaremos considerar diversos tipos de padrões ao fazer as funções anotadoras para a entidade de <mark>DIMENSAO</mark>.
</h4>

In [27]:
import re
## (PALAVRA) pode conter uma palavra | PALAVRA precisa conter uma palavra.
padroes = [
    # NUM X NUM X NUM (PALAVRA) -- 2.5 x 3 x 4 cm
    r'\b(\d+[\.,]?\d*)\s*x\s*(\d+[\.,]?\d*)\s*x\s*(\d+[\.,]?\d*)\s*([a-z]*)', 
    # NUM PALAVRA X NUM PALAVRA X NUM PALAVRA -- 2.5 cm x 3 m x 4 mm
    r'\b(\d+[\.,]?\d*)\s*([a-z]+)\s*x\s*(\d+[\.,]?\d*)\s*([a-z]+)\s*x\s*(\d+[\.,]?\d*)\s*([a-z]+)\b', 
    # NUM X NUM (PALAVRA) -- 2 x 3 cm
    r'\b(\d+[\.,]?\d*)\s*x\s*(\d+[\.,]?\d*)\s*([a-z]*)', 
    # NUM PALAVRA X NUM PALAVRA -- 2cm x 3m
    r'\b(\d+[\.,]?\d*)\s*([a-z]+)\s*x\s*(\d+[\.,]?\d*)\s*([a-z]+)\b',
    # NUM X NUM -- 2.5 x 3
    r'\b(\d+[\.,]?\d*)\s*x\s*(\d+[\.,]?\d*)\b',
    # NUM (m, cm, mm, metro, metros, pol) -- 2 metros | Mais restrito para evitar coletar número de série ou lixo nas descrições
    r'\b(\d+[\.,]?\d*)\s*(m|cm|mm|metro|metros|pol)\b'
]

padroes_compilados = [re.compile(padrao, re.IGNORECASE) for padrao in padroes]

<h4>
Vamos criar uma heurística para a dimensão, se um dos padrões acontece inferimos que aquele termo está representando uma dimensão.
</h4>

In [28]:
def detectar_dimensao(doc):
    text = doc.text
    for padrao in padroes_compilados:
        for match in padrao.finditer(text):
            start_char, end_char = match.span()
            span = doc.char_span(start_char, end_char, alignment_mode="expand")
            if span:
                yield span.start, span.end, "DIMENSAO"

In [29]:
from skweak import heuristics
dimensao_annotator = heuristics.FunctionAnnotator(
    "dimensao_annotator", 
    detectar_dimensao,
)

In [30]:
from skweak.gazetteers import GazetteerAnnotator, Trie
from skweak.base import CombinedAnnotator

In [31]:
especie_terms = []
for key, synonyms in madeiras_dicionario.items():
    especie_terms.append(key)
    especie_terms.extend(synonyms)

cientifico_especie_terms = []
for key, cientifico in cientifico_madeiras_dicionario.items():
    cientifico_especie_terms.extend(cientifico)

corte_terms = []
for key, synonyms in apresentacoes_dicionario.items():
    corte_terms.append(key)
    corte_terms.extend(synonyms)

In [32]:
def build_trie(term_list):
    """Convert a list of terms into a trie of token sequences"""
    trie = Trie()
    for term in term_list:
        tokens = term.split()
        trie.add(tokens)
    return trie

especie_trie = build_trie(especie_terms)
cientifico_especie_trie = build_trie(cientifico_especie_terms)
corte_trie = build_trie(corte_terms)

In [33]:
cientifico_especie_trie

[['vouacapoua'], ['vouacapoua', 'americana'], ['parahancornia'], ['parahancornia', 'amapa'], ['protium'], ['protium', 'heptaphyllum'], ['heptaphyllum'], ['carapa'], ['carapa', 'guianensis'], ['guianensis'], ['vatairea'], ['vatairea', 'sericea'], ['sericea'], ['hymenolobium'], ['hymenolobium', 'petraeum'], ['petraeum'], ['dinizia'], ['dinizia', 'excelsa'], ['excelsa'], ['erisma'], ['erisma', 'uncinatum'], ['uncinatum'], ['cedrela'], ['cedrela', 'fissilis'], ['fissilis'], ['amburana'], ['amburana', 'cearensis'], ['cearensis'], ['cedrelinga'], ['cedrelinga', 'cateniformis'], ['cateniformis'], ['orbygnia'], ['orbygnia', 'phalerata'], ['phalerata'], ['copaifera'], ['copaifera', 'langsdorffii'], ['langsdorffii'], ['dipteryx'], ['dipteryx', 'odorata'], ['odorata'], ['goupia'], ['goupia', 'glabra'], ['glabra'], ['eucalyptus'], ['cordia'], ['cordia', 'goeldiana'], ['goeldiana'], ['apuleia'], ['apuleia', 'leiocarpa'], ['leiocarpa'], ['qualea'], ['terminalia'], ['terminalia', 'amazonia'], ['amazo

In [34]:
especie_annotator = GazetteerAnnotator("especie_annotator", {"ESPECIE": especie_trie})
cientifico_especie_annotator = GazetteerAnnotator("cientifico_especie_annotator", {"ESPECIE": cientifico_especie_trie})
corte_annotator = GazetteerAnnotator("corte_annotator", {"CORTE": corte_trie})

In [35]:
combined = CombinedAnnotator()
combined.add_annotator(especie_annotator)
combined.add_annotator(cientifico_especie_annotator)
combined.add_annotator(corte_annotator)
combined.add_annotator(dimensao_annotator)

In [36]:
doc = nlp("madeira serrada massaranduba (manilkara huberi) 2.5 metro 2 vigas de 10x15cm e 3 tabuas de 2,5x30x200cm")
combined(doc)
skweak.utils.display_entities(doc, ["especie_annotator", "cientifico_especie_annotator", "corte_annotator", "dimensao_annotator"])

In [37]:
print("ESPECIE:", [ent.text for ent in doc.spans["especie_annotator"]])
print("ESPECIE:", [ent.text for ent in doc.spans["cientifico_especie_annotator"]])
print("CORTE:", [ent.text for ent in doc.spans["corte_annotator"]])
print("DIMENSAO:", [ent.text for ent in doc.spans["dimensao_annotator"]])

ESPECIE: ['massaranduba']
ESPECIE: ['manilkara huberi']
CORTE: ['vigas', 'tabuas']
DIMENSAO: ['2,5x30x200cm', '10x15cm', '2,5x30x200', '2.5 metro']


<h4>
Utilizando um exemplo do conjunto de treinamento.
</h4>

In [38]:
doc = spacy_docs_train[0]

combined(doc)

skweak.utils.display_entities(doc, ["especie_annotator", "cientifico_especie_annotator", "corte_annotator", "dimensao_annotator"])

In [39]:
from pydantic import BaseModel, Field
from typing import List, Optional, Sequence

class ESPECIE(BaseModel):
    nome: Optional[str] = Field(None, description="Nome comercial da madeira.")
    nome_cientifico: Optional[str] = Field(None, description="Nome científico da madeira")

class CORTE(BaseModel):
    nome: str = Field(..., description="Tipo do corte da madeira.")

class DIMENSAO(BaseModel):
    medida: str = Field(..., description="Medidas do produto de madeira.")

class MadeiraInfo(BaseModel):
    especies: Optional[List[ESPECIE]] = Field(default_factory=list, alias="ESPECIE")
    cortes: Optional[List[CORTE]] = Field(default_factory=list, alias="CORTE")
    dimensoes: Optional[List[DIMENSAO]] = Field(default_factory=list, alias="DIMENSAO")


In [40]:
system_prompt = """Você é um especialista em produtos de madeira serrada. Extraia entidades no seguinte formato JSON:
{
  "ESPECIE": [
    {"nome": "nome_comum", "nome_cientifico": "nome_cientifico"}
  ],
  "CORTE": [
    {"nome": "tipo_corte"}
  ],
  "DIMENSAO": [
    {"medida": "valor_dimensao"}
  ]
}
Regras:
- Mantenha os valores exatos do texto
- Use sempre arrays mesmo para único elemento
Dicionário para os campos:
- ESPECIE: 
  - nome: O nome comercial de um tipo de madeira. (ex.: massaranduba, pinus, angelim, jatoba, mogno)
  - nome_cientifico: O nome cientrifico de um tipo de madeira. (ex.: manilkara huberi, dinizia excelsa, swietenia macrophylla)
- CORTE: O formato ou tipo da peça de madeira. (ex.: viga, tabua, ripa, barrote, caibro)
- DIMENSOES: As dimensões da peça de madeira, representam o comprimento, largura e altura da peça. (ex.: 2.5x3x4cm, 2cm x 1.5cm x 1m, 2x3cm, 1cm x 2.5m, 2.5x3, 2 metros)
Exemplo entrada: 
"madeira massaranduba (manilkara huberi) em vigas 10x15cm"
Exemplo saída:
{
  "ESPECIE": [{"nome": "massaranduba", "nome_cientifico": "manilkara huberi"}],
  "CORTE": [{"nome": "vigas"}],
  "DIMENSAO": [{"medida": "10x15cm"}]}"""

In [41]:
# import ollama
# def extract_wood_entities(text: str) -> MadeiraInfo:
#     """Extract entities using Ollama with structured JSON output"""
#     response = ollama.chat(
#         model='qwen3:4b',
#         format='json',
#         messages=[
#             {
#                 'role': 'system',
#                 'content': system_prompt
#             },
#             {
#                 'role': 'user',
#                 'content': text
#             }
#         ],
#         options={'temperature': 0.3}
#     )
#     return MadeiraInfo.model_validate_json(response['message']['content'])

In [42]:
# def find_entity_spans(text: str, entities: MadeiraInfo) -> list[dict]:
#     """Find exact text spans for each entity"""
#     results = []
    
#     def add_matches(value: str, label: str):
#         if not value:
#             return
#         for match in re.finditer(re.escape(value.lower()), text.lower()):
#             start = text.lower().find(value.lower(), match.start(), match.end())
#             end = start + len(value)
#             results.append({
#                 'start': start,
#                 'end': end,
#                 'label': label,
#                 'text': text[start:end]
#             })
    
#     for especie in entities.especies:
#         add_matches(especie.nome, "ESPECIE")
#         add_matches(especie.nome_cientifico, "ESPECIE")
    
#     for corte in entities.cortes:
#         add_matches(corte.nome, "CORTE")
    
#     for dimensao in entities.dimensoes:
#         add_matches(dimensao.medida, "DIMENSAO")
    
#     seen = set()
#     unique_results = []
#     for res in results:
#         key = (res['start'], res['end'], res['label'])
#         if key not in seen:
#             seen.add(key)
#             unique_results.append(res)
    
#     return sorted(unique_results, key=lambda x: x['start'])

In [43]:

# text = "ripa de cedro 1cm x 5 cm 2 m"
# entities = extract_wood_entities(text)
# spans = find_entity_spans(text, entities)

# print("Extracted entities:")
# for span in spans:
#     print(f"{span['text']} ({span['label']}) [{span['start']}-{span['end']}]")

In [44]:
# from spacy import displacy

# # Define a dictionary to map entity labels to specific colors for visualization
# colors = {
#     "ESPECIE": "#2ECC71",  # Green color for 'Medicamento' entities
#     "CORTE": "#3498DB",  # Blue color for 'Procedimento' entities
#     "DIMENSAO": "#E74C3C",  # Red color for 'Doença' entities
# }

# # Define options for displacy visualization, including the entity labels and their corresponding colors
# options = {"ents": list(colors.keys()), "colors": colors}

# # Prepare the data for rendering by displacy
# # 'render_data' is a list of dictionaries, each containing the text and its corresponding entities
# render_data = [{"text": text, "ents": spans}]

# # Render the entities in the text using displacy with the specified options
# # 'style="ent"' specifies that we are visualizing named entities
# # 'manual=True' indicates that we are providing the entities manually
# # 'jupyter=True' ensures the visualization is displayed correctly in a Jupyter notebook
# displacy.render(render_data, style="ent", manual=True, jupyter=True, options=options)

In [45]:
# text = df_unlabeled.text.iloc[0]
# entities = extract_wood_entities(text)
# spans = find_entity_spans(text, entities)

# print("Extracted entities:")
# for span in spans:
#     print(f"{span['text']} ({span['label']}) [{span['start']}-{span['end']}]")

# render_data = [{"text": text, "ents": spans}]
# displacy.render(render_data, style="ent", manual=True, jupyter=True, options=options)

In [46]:
# from spacy.tokens import Doc

# def detectar_entidades_llm(doc: Doc):
#     text = doc.text
    
#     try:
#         entities = extract_wood_entities(text)
#         spans = find_entity_spans(text, entities)
        
#         for span in spans:
#             char_span = doc.char_span(
#                 span['start'], 
#                 span['end'],
#                 label=span['label'],
#                 alignment_mode="expand"
#             )
            
#             if char_span and isinstance(char_span.start, int) and isinstance(char_span.end, int):
#                 yield (char_span.start, char_span.end, span['label'])
                
#     except Exception as e:
#         print(f"Error processing document: {e}")
#         return

In [47]:
# llm_annotator = heuristics.FunctionAnnotator(
#     "llm_annotator",
#     detectar_entidades_llm
# )

In [48]:
# combined.add_annotator(llm_annotator)

In [49]:
# text = df_unlabeled.text.iloc[0]
# doc = nlp(text)
# combined(doc)
# skweak.utils.display_entities(doc, ["especie_annotator", "cientifico_especie_annotator", "corte_annotator", "dimensao_annotator", "llm_annotator"])

## Usando Gpt

In [57]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
import os

load_dotenv(
    override=True
)

api_key_preview = os.getenv("OPENAI_API_KEY")[:5]
print(f"First 5 characters of API key: {api_key_preview}")

First 5 characters of API key: sk-pr


In [58]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Você é um especialista em extração de entidades nomeadas com precisão excepcional. "
            "Sua tarefa é identificar e extrair informações específicas do texto fornecido, seguindo estas diretrizes:"
            "\n\n1. Extraia as informações exatamente como aparecem no texto, sem interpretações ou alterações."
            "\n2. Se uma informação solicitada não estiver presente ou for ambígua, retorne null para esse campo."
            "\n3. Mantenha-se estritamente dentro do escopo das entidades e atributos definidos no esquema fornecido."
            "\n4. Preste atenção especial para manter a mesma ortografia, pontuação e formatação das informações extraídas."
            "\n5. Não infira ou adicione informações que não estejam explicitamente presentes no texto."
            "\n6. Se houver múltiplas menções da mesma entidade, extraia todas as ocorrências relevantes."
            "\n7. Ignore informações irrelevantes ou fora do contexto das entidades solicitadas."
            "\n\nLembre-se: sua precisão e aderência ao texto original são cruciais para o sucesso desta tarefa.",
        ),
        ("human", "{text}"),
    ]
)

In [59]:
model_openai = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# model_openai.invoke("Oi!")

In [60]:
model_openai = prompt | model_openai.with_structured_output(
    MadeiraInfo, include_raw=False, method = "function_calling",
).with_retry(
    stop_after_attempt=10,  # Retry up to 10 times in case of failure
    wait_exponential_jitter=True,  # Add randomness to the wait time between retries
)

In [40]:
# result = model_openai.invoke({"text": df_unlabeled.text.iloc[0]})
# print(result)

In [61]:
from tqdm import tqdm
batch_size = 100
texts = df_unlabeled["text"].tolist()
text_batches = [texts[i : i + 100] for i in range(0, len(texts), 100)]

# run the model
results = []
for batch in tqdm(text_batches):
    try:
        out = model_openai.batch(batch, config={"max_concurrency": 100})
        results.extend(out)
    except Exception as e:
        print(e)
        results.extend([None] * len(batch))

100%|██████████| 1/1 [00:08<00:00,  8.44s/it]


In [67]:
results[98]

MadeiraInfo(especies=[ESPECIE(nome='massar', nome_cientifico='null')], cortes=[CORTE(nome='caibro')], dimensoes=None)

In [63]:
type(results)

list

In [ ]:
from typing import Any

def convert_pydantic_to_gliner_format(
    text: str, madeira_info: MadeiraInfo  # Changed parameter name and type
) -> List[dict[str, Any]]:
    """
    Convert wood product data to GLiNER format.

    Args:
        text (str): Original text.
        wood_info (MadeiraInfo): Pydantic model with wood product information.

    Returns:
        List[Dict[str, Any]]: List of entity dictionaries in GLiNER format.
    """
    result = []
    if wood_info is None:
        return []

    # Handle all three entity types with their respective labels
    entity_mapping = [
        (wood_info.especies, ["nome", "nome_cientifico"], ["especie", "especie_cientifica"]),
        (wood_info.cortes, ["nome"], ["corte"]),
        (wood_info.dimensoes, ["medida"], ["dimensao"])
    ]

    for entity_list, fields, labels in entity_mapping:
        if entity_list is None:
            continue
            
        for entity in entity_list:
            for field, label in zip(fields, labels):
                value = getattr(entity, field)
                if value:
                    # Find all matches in the text
                    for match in re.finditer(re.escape(value), text, re.IGNORECASE):
                        result.append({
                            "start": match.start(),
                            "end": match.end(),
                            "text": match.group(),
                            "label": label,
                            "score": 1.0
                        })

    # Sort by start position
    result.sort(key=lambda x: x["start"])
    return result

In [69]:
df_lf_openai = pd.DataFrame(texts, columns=["text"])
df_lf_openai["weak_label"] = results


In [ ]:
# df_lf_openai.to_csv('100_lf_openai.csv', sep = ';', index=False)

In [72]:
df_lf_openai["weak_label"] = df_lf_openai.apply(
    lambda x: convert_pydantic_to_gliner_format(x.text, x.weak_label), axis=1
)

df_lf_openai

AttributeError: 'MadeiraInfo' object has no attribute 'medicamentos'

## 2.Gliner

<h4>
Além dos anotadores que criamos com a lista de espécies e corte de madeira, e por o problema ser bem específico não fui capaz de achar nenhum tipo de modelo pré-treinado para a tarefa de NER do domínio selecionado.<br>
Por isso vamos utilizar os modelos zero shot do GliNER, utilizando os nossos domínios esses modelos irão tentar rotular o texto sem nenhum treinamento prévio, o que pode ajudar bastante a solução da tarefa.  
</h4>

In [42]:
# from gliner import GLiNER

# model_gliner_llama = GLiNER.from_pretrained(
#     "knowledgator/gliner-llama-1.3B-v1.0",
# )

# model_gliner_bi_large = GLiNER.from_pretrained(
#     "knowledgator/gliner-bi-large-v1.0",
# )

# model_gliner_qwen = GLiNER.from_pretrained(
#     "knowledgator/gliner-qwen-1.5B-v1.0",
# )

# model_gliner_llama = model_gliner_llama.half()
# model_gliner_bi_large = model_gliner_bi_large.half()
# model_gliner_qwen = model_gliner_qwen.half()

In [43]:
doc = nlp("madeira serrada massaranduba (manilkara huberi) 2.5 metro 2 vigas de 10x15cm e 3 tabuas de 2,5x30x200cm")
text = doc.text

labels = ["ESPECIE", "CORTE", "DIMENSAO"]

In [44]:
from spacy import displacy
from typing import Any

def render_entity_data_from_pipeline(
    text: str,
    pipeline_results: list[dict[str, Any]],
    colors: dict[str, str] = None,
    label_key_name: str = "entity_group",
) -> None:
    # Extract entity spans (start, end, label) from pipeline results
    entity_spans = [
        (result["start"], result["end"], result[label_key_name])
        for result in pipeline_results
    ]

    # If no colors are provided, assign default colors to each entity type
    if colors is None:
        entity_types = list(set(span[2] for span in entity_spans))
        # Default colors for up to 5 entity types; cycle if more
        default_colors = ['#FF0000', '#00FF00', '#0000FF', '#FFFF00', '#FF00FF']
        colors = {}
        for i, entity_type in enumerate(entity_types):
            colors[entity_type] = default_colors[i % len(default_colors)]

    # Configure displacy options with entity labels and their colors
    displacy_options = {
        "ents": list(colors.keys()),
        "colors": colors  # Pass the color mapping dictionary
    }

    # Prepare data in the format required by displacy
    displacy_data = [{
        "text": text,
        "ents": [
            {"start": start, "end": end, "label": label}
            for start, end, label in entity_spans
        ]
    }]

    # Render the entities using displacy
    displacy.render(
        displacy_data,
        style="ent",
        manual=True,
        jupyter=True,
        options=displacy_options
    )

In [45]:
# result = model_gliner_llama.predict_entities(text, labels, threshold=0.5)

# print(result)

# render_entity_data_from_pipeline(text, result, label_key_name="label")

In [46]:
# result = model_gliner_bi_large.predict_entities(text, labels, threshold=0.5)

# print(result)

# render_entity_data_from_pipeline(text, result, label_key_name="label")

In [47]:
# result = model_gliner_qwen.predict_entities(text, labels, threshold=0.5)

# print(result)

# render_entity_data_from_pipeline(text, result, label_key_name="label")

<h4>
Okay não começamos bem. Para o nosso exemplo simples apenas o modelo do Qwen foi capaz de identicar algumas entidades, parece que a entidade de corte foi um desafio para esses modelos.<br>
Vamos observar em uma quantidade maior de dados, vamos utilizar 50 instâncias do conjunto de treinamento para ver se podemos utilizar esses modelos como anotadores para o nosso problema.
</h4>

In [48]:
texts_to_test = [doc.text for doc in spacy_docs_train[:50]]

In [49]:
from tqdm import tqdm
def create_entity_dataframe(texts, model, labels, threshold=0.5):
    """
    Process multiple texts and create a DataFrame with entities
    
    Args:
        texts (list): List of texts to process
        model: Your GLiNER model
        labels (list): Entity labels to predict
        threshold (float): Confidence threshold
    
    Returns:
        pd.DataFrame: DataFrame with text, entities, and entity count
    """
    data = []
    
    for text in tqdm(texts):  # tqdm adds progress bar
        # Get predictions from model
        entities = model.predict_entities(text, labels, threshold=threshold)
        
        # Format results
        data.append({
            "text": text,
            "entities": entities,
            "n_entities": len(entities)
        })
    
    return pd.DataFrame(data)

In [50]:
# llama_df = create_entity_dataframe(
#     texts=texts_to_test,
#     model=model_gliner_llama,
#     labels=labels,
#     threshold=0.5
# )

<h4>
Contando a quantidade de entidades que o Llama foi capaz de reconhecer.
</h4>

In [51]:
# llama_df.n_entities.sum()

<h4>
O llama não foi capaz de achar entidades na amostra de 50 instâncias do conjunto de dados não rotulados, portanto já podemos eliminar o uso desse modelo da nossa pipeline.
</h4>

In [52]:
# bi_large_df = create_entity_dataframe(
#     texts=texts_to_test,
#     model=model_gliner_bi_large,
#     labels=labels,
#     threshold=0.5
# )

In [53]:
# bi_large_df.n_entities.sum()

In [54]:
# bi_large_entities = bi_large_df[bi_large_df.n_entities > 0]
# bi_large_entities[['text', 'entities']]

<h4>
O modelo bi_large foi capaz de achar 3 entidades, porém todas com a entidade errada, também podemos remover esse modelo da pipeline.
</h4>

In [55]:
# qwen_df = create_entity_dataframe(
#     texts=texts_to_test,
#     model=model_gliner_qwen,
#     labels=labels,
#     threshold=0.5
# )

In [56]:
# qwen_df.n_entities.sum()

In [57]:
# qwen_entities = qwen_df[qwen_df.n_entities > 0]
# qwen_entities[['text', 'entities']]

<h4>
O qwen foi o melhor dos 3 modelos, além de ter achado a maior quantidade de entidades grande parte delas está correta, porém existem alguns erros preocupantes, o texto da linha 12 foi completamente selecionada, realmente existe um CORTE, na descrição mas ele não foi devidamente selecionado, além disso Muiracatiara na linha 17 foi selecionado como corte, mas deveria ser uma espécie.<br>
Vamos observar os casos em que o modelo não selecionou nenhuma entidade para analisar se estamos tendo um bom coverage ao menos do rótulo desses textos.
</h4>

In [58]:
# qwen_not_entities = qwen_df[qwen_df.n_entities == 0]
# qwen_not_entities[['text', 'entities']]

<h4>
No conjunto de texto em que o Qwen não foi capaz de identificar nenhuma das entidades existem exemplos como a linha 8, 9, 22 que realmente não deveriam ser rotulados, porém a grande maioria deveria.<br>
Por esse motivo e o grande custo que esses modelos possuem comparados as funções de rotulagem que criamos antes também não irei utilizar o Zero shot do Qwen como uma função rotuladora.
</h4>

## 3. Aplicando Funções de Agregação

In [59]:
spacy_docs_train = list(combined.pipe(spacy_docs_train))

<h4>
Salvar o objeto do Spacy para não precisar rodar o processo anterior múltiplas vezes.
</h4>

In [60]:
skweak.utils.docbin_writer(
    spacy_docs_train, "artefatos/spacy_docs_train_annotated.bin"
)

Write to artefatos/spacy_docs_train_annotated.bin...done


# Save Point

In [61]:
import skweak

# Now we can load them back, so we don't need to run the labelling functions again
spacy_docs_train = skweak.utils.docbin_reader(
    "artefatos/spacy_docs_train_annotated.bin", spacy_model_name="pt_core_news_lg"
)
spacy_docs_train = list(spacy_docs_train)

In [62]:
majority_voter = skweak.aggregation.MajorityVoter(
    name="doclevel_voter",
    labels=["ESPECIE", "CORTE", "DIMENSAO"],
    initial_weights={"doc_majority": 0.0}  # Evitar a inclusão dele mesmo
)

spacy_docs_train = list(majority_voter.pipe(spacy_docs_train))

doc_majority = skweak.doclevel.DocumentMajorityAnnotator("doc_majority","doclevel_voter")

spacy_docs_train = list(doc_majority.pipe(spacy_docs_train))

In [63]:
initial_weights = {
    "especie_annotator": .9,
    "cientifico_especie_annotator": 1,
    "corte_annotator": 1,
    "dimensao_annotator": 1,
    "llm_annotator": 0.7,   
    "doclevel_voter": 0.5,
    "doc_majority": 0.6
}

hmm = skweak.generative.HMM(
    "hmm", 
    labels=["ESPECIE", "CORTE", "DIMENSAO"],
    initial_weights=initial_weights
)


hmm.fit(spacy_docs_train)

Starting iteration 1
Number of processed documents: 1000
Number of processed documents: 2000
Number of processed documents: 3000
Number of processed documents: 4000
Number of processed documents: 5000
Number of processed documents: 6000
Number of processed documents: 7000
Number of processed documents: 8000
Number of processed documents: 9000
Number of processed documents: 10000
Number of processed documents: 11000
Number of processed documents: 12000
Number of processed documents: 13000
Number of processed documents: 14000
Number of processed documents: 15000
Number of processed documents: 16000
Number of processed documents: 17000
Number of processed documents: 18000
Number of processed documents: 19000
Finished E-step with 19968 documents
Starting iteration 2


         1 -132771.27819403             +nan


Number of processed documents: 1000
Number of processed documents: 2000
Number of processed documents: 3000
Number of processed documents: 4000
Number of processed documents: 5000
Number of processed documents: 6000
Number of processed documents: 7000
Number of processed documents: 8000
Number of processed documents: 9000
Number of processed documents: 10000
Number of processed documents: 11000
Number of processed documents: 12000
Number of processed documents: 13000
Number of processed documents: 14000
Number of processed documents: 15000
Number of processed documents: 16000
Number of processed documents: 17000
Number of processed documents: 18000
Number of processed documents: 19000
Finished E-step with 19968 documents
Starting iteration 3


         2 -127659.25208165   +5112.02611238


Number of processed documents: 1000
Number of processed documents: 2000
Number of processed documents: 3000
Number of processed documents: 4000
Number of processed documents: 5000
Number of processed documents: 6000
Number of processed documents: 7000
Number of processed documents: 8000
Number of processed documents: 9000
Number of processed documents: 10000
Number of processed documents: 11000
Number of processed documents: 12000
Number of processed documents: 13000
Number of processed documents: 14000
Number of processed documents: 15000
Number of processed documents: 16000
Number of processed documents: 17000
Number of processed documents: 18000
Number of processed documents: 19000
Finished E-step with 19968 documents
Starting iteration 4


         3 -126179.63493980   +1479.61714186


Number of processed documents: 1000
Number of processed documents: 2000
Number of processed documents: 3000
Number of processed documents: 4000
Number of processed documents: 5000
Number of processed documents: 6000
Number of processed documents: 7000
Number of processed documents: 8000
Number of processed documents: 9000
Number of processed documents: 10000
Number of processed documents: 11000
Number of processed documents: 12000
Number of processed documents: 13000
Number of processed documents: 14000
Number of processed documents: 15000
Number of processed documents: 16000
Number of processed documents: 17000
Number of processed documents: 18000
Number of processed documents: 19000
Finished E-step with 19968 documents


         4 -125446.25398991    +733.38094989


In [64]:
hmm.pretty_print()

HMM model with following parameters:
Output labels: ['O', 'B-ESPECIE', 'I-ESPECIE', 'B-CORTE', 'I-CORTE', 'B-DIMENSAO', 'I-DIMENSAO']
--------
Start distribution:
O             0.49
B-ESPECIE     0.16
I-ESPECIE     0.00
B-CORTE       0.35
I-CORTE       0.00
B-DIMENSAO    0.00
I-DIMENSAO    0.00
dtype: float64
--------
Transition model:
               O  B-ESPECIE  I-ESPECIE  B-CORTE  I-CORTE  B-DIMENSAO  I-DIMENSAO
O           0.78       0.03       0.00     0.15      0.0        0.05        0.00
B-ESPECIE   0.64       0.00       0.17     0.00      0.0        0.19        0.00
I-ESPECIE   0.86       0.00       0.00     0.01      0.0        0.14        0.00
B-CORTE     0.67       0.21       0.00     0.01      0.0        0.12        0.00
I-CORTE     0.20       0.20       0.00     0.20      0.2        0.20        0.00
B-DIMENSAO  0.03       0.00       0.00     0.00      0.0        0.01        0.96
I-DIMENSAO  0.41       0.03       0.00     0.00      0.0        0.17        0.39
--------
Label

In [65]:
maj_voter = skweak.voting.SequentialMajorityVoter(
    "maj_voter",
    labels=["ESPECIE", "CORTE", "DIMENSAO"],
    initial_weights=initial_weights
)


In [66]:
spacy_docs_train = list(hmm.pipe(spacy_docs_train))

In [67]:
spacy_docs_train = list(maj_voter.pipe(spacy_docs_train))

In [68]:
def render_skweak_entities(doc, annotator_name: str, colors: dict = None):
    """Render entities from skweak annotations with colored highlighting"""
    
    # Get entities from specified annotator
    entities = doc.spans.get(annotator_name, [])
    
    # Default color scheme for your entities
    default_colors = {
        "ESPECIE": "#7FFFD4",  # Aquamarine
        "CORTE": "#FFD700",    # Gold
        "DIMENSAO": "#FFB6C1"  # Pink
    }
    
    # Ensure colors is never None
    colors = colors or default_colors
    
    # Prepare entities for visualization
    displacy_data = {
        "text": doc.text,
        "ents": [
            {
                "start": ent.start_char,
                "end": ent.end_char,
                "label": ent.label_
            } for ent in entities
        ],
        "title": None
    }

    # Render with specified colors (ensure empty dict if no colors)
    displacy.render(
        [displacy_data],
        style="ent",
        manual=True,
        jupyter=True,
        options={"colors": colors or {}}  # Key fix here
    )

# Usage
doc = spacy_docs_train[0]

print("=== HMM Annotations ===")
render_skweak_entities(doc, "hmm")


print("=== Majority Voter Annotations ===")
render_skweak_entities(doc, "maj_voter")

=== HMM Annotations ===


=== Majority Voter Annotations ===


<h4>
Se
</h4>

In [69]:
docs_sem_entidades = [
    doc for doc in spacy_docs_train[:50] 
    if len(doc.spans.get("hmm", [])) == 0
]

df_sem_entidades_hmm = pd.DataFrame({
    "text": [doc.text for doc in docs_sem_entidades],
    "num_entidades": [len(doc.spans.get("hmm", [])) for doc in docs_sem_entidades]
})

df_sem_entidades_maj_voter = pd.DataFrame({
    "text": [doc.text for doc in docs_sem_entidades],
    "num_entidades": [len(doc.spans.get("maj_voter", [])) for doc in docs_sem_entidades]
})

print(len(df_sem_entidades_hmm))
print(len(df_sem_entidades_maj_voter))

4
4


In [70]:
df_sem_entidades_hmm.head()

,text,num_entidades
0,jogo cozinha em madeira,0
1,conjunto de utensilios para cozi,0
2,palitos p/ dentes aurea gde,0
3,pv-kit colher / espatula em made,0


In [71]:
df_sem_entidades_maj_voter.head()

,text,num_entidades
0,jogo cozinha em madeira,0
1,conjunto de utensilios para cozi,0
2,palitos p/ dentes aurea gde,0
3,pv-kit colher / espatula em made,0


<h4>
Parece que ambos agregadores conseguiram explorar bem o conjunto de dados e identificar ao menos uma entidade quando o produto era realmente de madeira serrada.
</h4>

In [72]:
from skweak.utils import docbin_writer
docbin_writer(spacy_docs_train, "artefatos/spacy_docs_train_annotated.bin")

Write to artefatos/spacy_docs_train_annotated.bin...done


## 4. Utilizar as Weak Labels para produzir um novo modelo.

<h4>
Agora preciso utilizar as funções de rotulagem que implementei até agora para fazer o fine-tuning de um modelo Bert pré-treinado capaz de generalizar as regras 'hard' do Gazetteer e da heurística que criei, além das funções agregadoras.
</h4>

In [ ]:
from skweak.utils import docbin_reader

spacy_docs_train = list(docbin_reader("artefatos/spacy_docs_train_annotated.bin", 
                                     spacy_model_name="pt_core_news_lg"))

doc = spacy_docs_train[0]
print("Entidades:", [(ent.text, ent.label_) for ent in doc.spans["hmm"]])

In [ ]:
def convert_to_iob(docs, annotator_name="hmm"):
    """Convert skweak annotations to IOB format"""
    iob_data = []
    for doc in docs:
        tokens = [token.text for token in doc]
        labels = ["O"] * len(tokens)
        
        # Process each entity span
        for span in doc.spans[annotator_name]:
            label = span.label_
            start = span.start
            end = span.end
            
            labels[start] = f"B-{label}"
            for i in range(start+1, end):
                labels[i] = f"I-{label}"
                
        iob_data.append({"tokens": tokens, "labels": labels, "text": doc.text})
    
    return iob_data

train_data = convert_to_iob(spacy_docs_train, annotator_name="hmm")

In [ ]:
import ast
def convert_argilla_to_iob(argilla_data):
    """Convert Argilla format to IOB format"""
    iob_data = []
    
    for example in argilla_data:
        text = example["text"]
        entities = example.get("span_label.responses", [])
        
        if isinstance(entities, str):
            try:
                clean_str = entities.replace("array(", "").replace(", dtype=object)", "")
                entities = ast.literal_eval(clean_str)
            except:
                entities = []

        doc = nlp(text)
        tokens = [token.text for token in doc]
        labels = ["O"] * len(tokens)

        for entity in entities:
            if not isinstance(entity, dict):
                continue

            start = entity["start"]
            end = entity["end"]
            label = entity["label"]

            start_token = None
            for token in doc:
                if token.idx >= start:
                    break
                start_token = token.i
            if start_token is None:
                start_token = 0

            end_token = None
            for token in doc:
                if token.idx + len(token) > end:
                    break
                end_token = token.i
            if end_token is None:
                end_token = len(doc) - 1

            if start_token <= end_token:
                labels[start_token] = f"B-{label}"
                for i in range(start_token + 1, end_token + 1):
                    labels[i] = f"I-{label}"

        iob_data.append({
            "tokens": tokens,
            "labels": labels,
            "text": text
        })
    
    return iob_data

val_data = convert_argilla_to_iob(df_val.to_dict("records"))
test_data = convert_argilla_to_iob(df_test.to_dict("records"))

In [ ]:
from datasets import Dataset, Features, Sequence, ClassLabel, Value

labels = ["O", "B-ESPECIE", "I-ESPECIE", "B-CORTE", "I-CORTE", "B-DIMENSAO", "I-DIMENSAO"]

dataset_train = Dataset.from_list(train_data).cast(Features({
    "tokens": Sequence(Value("string")),
    "labels": Sequence(ClassLabel(names=labels)),
    "text": Value("string")
}))
dataset_val = Dataset.from_list(val_data).cast(Features({
    "tokens": Sequence(Value("string")),
    "labels": Sequence(ClassLabel(names=labels)),
    "text": Value("string")
}))
dataset_test = Dataset.from_list(test_data).cast(Features({
    "tokens": Sequence(Value("string")),
    "labels": Sequence(ClassLabel(names=labels)),
    "text": Value("string")
}))



In [ ]:
from transformers import AutoTokenizer, DataCollatorForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], 
        truncation=True, 
        is_split_into_words=True,
        max_length=512
    )

    labels = []
    for i, label in enumerate(examples["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []

        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_dataset_train = dataset_train.map(tokenize_and_align_labels, batched=True)
tokenized_dataset_val = dataset_val.map(tokenize_and_align_labels, batched=True)
tokenized_dataset_test = dataset_test.map(tokenize_and_align_labels, batched=True)

data_collator = DataCollatorForTokenClassification(tokenizer, padding=True, max_length=512, return_tensors="pt")


In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    "neuralmind/bert-base-portuguese-cased",
    num_labels=len(labels),
    id2label={i: label for i, label in enumerate(labels)},
    label2id={label: i for i, label in enumerate(labels)}
)

training_args = TrainingArguments(
    output_dir="./artefatos/bert-base-ner-true-labels",  # Directory to save model checkpoints and logs
    num_train_epochs=5,  # Number of training epochs
    per_device_train_batch_size=128,  # Batch size for training
    per_device_eval_batch_size=64,  # Batch size for evaluation
    gradient_accumulation_steps=1,  # Accumulate gradients over multiple steps
    weight_decay=0.01,  # Weight decay for regularization
    seed=271828,  # Random seed for reproducibility
    bf16=True,  # Use bfloat16 precision for training (if supported by hardware)
    fp16=False,  # Use mixed precision training with FP16 (if supported by hardware)
    save_total_limit=1,  # Limit the total number of saved checkpoints
    logging_steps=1,  # Log training metrics every step
    eval_steps=1,  # Evaluate the model every step
    save_steps=1,  # Save the model every step
    metric_for_best_model="eval_loss",  # Metric to determine the best model
    greater_is_better=False,  # Higher metric value is better
    logging_strategy="steps",  # Log metrics at each step
    eval_strategy="epoch",  # Evaluate the model at the end of each epoch
    save_strategy="epoch",  # Save the model at the end of each epoch
    load_best_model_at_end=True,  # Load the best model at the end of training
    do_train=True,  # Perform training
    do_eval=True,  # Perform evaluation
    push_to_hub=False,  # Do not push the model to the Hugging Face Hub
    learning_rate=1e-5,  # Learning rate for the optimizer
    overwrite_output_dir=True,  # Overwrite the output directory if it exists
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_val,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()

<h4>
Parece que o modelo não foi capaz de generalizar. Mesmo após fazer alguns testes modificando os parâmteros de treino, como o learning rate, per_device_train_batch_size, weight_decay, eu não fui capaz de encontrar uma combinação que diminuísse a Validation Loss.<br>
Mesmo assim vamos analizar o comportamento desse modelo com alguns exemplos e no nosso conjunto de teste.
</h4>

In [ ]:
from transformers import pipeline

ner_pipeline = pipeline(
    "ner", 
    model=model,
    tokenizer=tokenizer,
    aggregation_strategy="simple"
)

text = "angelim verm aproveitamento ripas" ## Exemplo do conjunto de treino, errou em ri ## pas
results = ner_pipeline(text)

print("Entities detected:")
for entity in results:
    print(f"{entity['word']} -> {entity['entity_group']}")

In [ ]:
textos_teste = tokenized_dataset_test['text'][:50]

In [ ]:
results_list = ner_pipeline(textos_teste)

In [ ]:
print("Entities detected:")
for entity in results_list[0]:
    print(f"{entity['word']} -> {entity['entity_group']}")

In [ ]:
print("Entities detected:")
for entity in results_list[1]:
    print(f"{entity['word']} -> {entity['entity_group']}")

## Objetivo final

Analytics & Market Intelligence

Use case: Aggregate and analyze trends across product descriptions.

Why it matters:

    What species are trending?

    Is there increased demand for rift-sawn oak in certain dimensions?

    Your model enables text mining on raw product text to derive insights.

Structured Product Catalogs for E-commerce Platforms

Use case: Automatically structuring unstructured product descriptions into standardized fields.

Why it matters:

    Timber suppliers often have messy, inconsistent listings (e.g., “Oak rift sawn board 2x6” vs. “2x6 white oak, rift cut”).

    Your NER model can power a data normalization pipeline that extracts:

        Species: white oak, Douglas fir

        Cut type: plain sawn, quarter sawn, rift sawn

        Dimensions: 2x6, 25mm x 200mm

Outcome: Cleaner databases, improved search/filter/sorting, better inventory management.